In [ ]:
import time
import requests
import pyautogui
import pyperclip
import os
import pandas as pd
from openpyxl import load_workbook
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.edge.options import Options 
from selenium.webdriver.common.by import By
from time import sleep
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from PIL import Image
import win32clipboard
from io import BytesIO
from datetime import datetime
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import locale

def loadingPage():
    # Aguarda até que o estado do documento seja 'complete'
    while driver.execute_script("return document.readyState") != "complete":
        pass

def loadingElement(tipo, elemento):
    elemento = WebDriverWait(driver, 60).until(
        EC.presence_of_element_located((tipo, elemento))
    )

MAX_TENTATIVAS = 3

def verificarPaginaErro():
    """Verifica se a página de erro do Teams foi carregada."""
    try:
        erro = driver.find_element(By.XPATH, '/html/body/div/div/div[1]')
        print("Página de erro detectada. Recarregando...")
        return True
    except NoSuchElementException:
        return False

def loginTeams(tentativas=0):
    """Executa o login no Microsoft Teams e trata possíveis erros."""
    if tentativas >= MAX_TENTATIVAS:
        print("Máximo de tentativas alcançado. Verifique a conexão ou a página do Teams.")
        return

    #print(f"Tentativa {tentativas + 1} de login...")
    driver.get('url-login')

    try:
        # Aguarda a presença do botão de login por até 30 segundos.
        botao_login = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.XPATH, "//small[contains(text(), 'email-selecionado')]"))
        )
        botao_login.click()

        # Aguarda até que a URL da página principal do Teams esteja correta.
        WebDriverWait(driver, 60).until(
            lambda d: d.current_url == 'https://teams.microsoft.com/v2/'
        )
        #print("Login realizado com sucesso!")

    except TimeoutException:
        print("TimeoutException detectada.")
        if verificarPaginaErro():
            driver.refresh()  # Recarrega a página se for detectada a página de erro.
            WebDriverWait(driver, 30).until(
                EC.presence_of_element_located((By.TAG_NAME, 'body'))  # Aguarda a página carregar.
            )
            loginTeams(tentativas + 1)  # Tenta o login novamente.
        else:
            print("Timeout sem página de erro. Verifique sua conexão ou o script.")

def buscaChat(CssChat):
    max_tentativas = 3
    tentativa_atual = 0
    chat_encontrado = False
    
    while tentativa_atual < max_tentativas and not chat_encontrado:
        try:
            loadingElement(By.XPATH,'/html/body/script[8]')
            loadingElement(By.CSS_SELECTOR, CssChat)
            # Aguarda até que o chat esteja clicável
            chatpausas = WebDriverWait(driver, 30).until(
                EC.element_to_be_clickable(( By.CSS_SELECTOR, CssChat))
            )                                                 
            chatpausas.click()
            chat_encontrado = True  # Sai do loop se o chat for encontrado e clicado com sucesso
            #print(f"Chat encontrado na tentativa {tentativa_atual + 1}")
        except TimeoutException:
            page= driver.current_url
            if page != 'https://teams.microsoft.com/v2/':
                driver.get('https://teams.microsoft.com/v2/')
            tentativa_atual += 1
            print(f"Chat não encontrado na tentativa {tentativa_atual}. Atualizando a página.")
            driver.refresh()  # Atualiza a página para tentar novamente
            loadingPage()
    if not chat_encontrado:
        print("Não foi possível encontrar o chat após várias tentativas.")
        #driver.quit()  # Fecha o driver se não conseguir encontrar o chat

def colarImagem():
    caixa_de_texto = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[contains(@id, 'new-message')]")))
    caixa_de_texto.click()
    image= Image.open('printMonitor.png') # abre print         
    # Converter a imagem para BMP em um buffer de memória
    output = BytesIO()
    image.convert("RGB").save(output, "BMP")
    data = output.getvalue()[14:]
    output.close()
    # Copiar a imagem para a área de transferência
    win32clipboard.OpenClipboard()
    win32clipboard.EmptyClipboard()
    win32clipboard.SetClipboardData(win32clipboard.CF_DIB, data)
    win32clipboard.CloseClipboard()
    action.key_down(Keys.CONTROL).send_keys('v').key_up(Keys.CONTROL).perform()
    time.sleep(5)
    caixa_de_texto.send_keys('\n')

def exibir_data_hora_atual():
    data_hora_atual = datetime.now()
    data_hora_formatada = data_hora_atual.strftime("%Y-%m-%d %H:%M:%S")
    print("Data e hora atuais:", data_hora_formatada)
    
def exibir_data_hora_pausa():
    # Define o locale para português do Brasil
    locale.setlocale(locale.LC_TIME, 'Portuguese_Brazil.1252')

    # Obtém a data e hora atuais
    data_hora_atual = datetime.now()
    
    # Formata o dia da semana, data e hora
    dia_da_semana = data_hora_atual.strftime("%A")  # Nome do dia da semana em português
    data_formatada = data_hora_atual.strftime("%d-%m-%Y")  # Data no formato DD-MM-YYYY
    hora_formatada = data_hora_atual.strftime("%H:%M:%S")  # Hora no formato HH:MM:SS
    
    # Retorna o resultado formatado
    return f"{dia_da_semana.capitalize()}, {data_formatada}, {hora_formatada}"

while True:
    # Tentativa de clicar no chat
    max_tentativas = 3
    tentativa_atual = 0
    chat_encontrado = False
    # hora atual
    agora = datetime.now().time()
    
    # Definir os horários limite
    hora_inicio = datetime.strptime("08:00", "%H:%M").time()
    hora_fim = datetime.strptime("23:59", "%H:%M").time()
    
    # Verificar se o horário atual não está entre 00:00 e 08:00
    if  (hora_inicio <= agora <= hora_fim):
        
        for check_relatorio in range(2):
            # Inicialize a lista de argumentos
            args = [
                '--headless', 'window-size=1920,1080', 
            ]
            # Configurar as opções do WebDriver
            options = Options()
            for arg in args:
                options.add_argument(arg)
            
            # Inicialize o WebDriver usando webdriver-manager
            driver = webdriver.Edge(options=options)  
            action = ActionChains(driver)
            
            # monitor
            driver.get('http://link-site-base')
            loadingPage()
            loadingElement(By.ID, 'email')
            input_email= driver.find_element( By.ID, 'email').send_keys('user')
            sleep(0.5)
            input_password = driver.find_element(By.ID, 'password').send_keys('password')
            sleep(0.5)
            button_login= driver.find_element(By.XPATH, '/html/body/div/div[2]/div/form/div[3]/div[2]/button').click()
            loadingPage()
            loadingElement(By.XPATH, '//*[@id="dash1_filter"]/label/input')
            filtro_pausas= driver.find_element(By.XPATH, '//*[@id="dash1_filter"]/label/input').send_keys('PAUSA') # filtro de pausas
            time.sleep(3)
            dados = driver.find_element(By.CSS_SELECTOR,'#agent_resume > tbody').text  #numeros de agentes ativos,pausa,etc
            dado = dados.split() #transforma em lista
            operadores = driver.find_element(By.CSS_SELECTOR,'#dash1 > tbody').text # todas informacoes da tabela do monitor
            filtro_opon = str(operadores.split('\n')) # lista de operadores online
            op_on = filtro_opon.split() # transforma em lista 

            # Certifique-se de que é um número antes de converter
            if dado[2].isdigit():
                ag_disponiveis = int(dado[2])
            else:
                # caso em que dado[2] não é um número válido
                ag_disponiveis = 0  
            # Pronto
            if dado[3].isdigit():
                ag_pronto = int(dado[3])
            else:
                ag_pronto = 0                  
            if dado[4].isdigit():
                ag_pausa = int(dado[4])
            else:
                ag_pausa = 0                                  
            # Ocupado
            if dado[5].isdigit():
                ag_ocupado = int(dado[5])
            else:                
                ag_ocupado = 0                 
            # Não Pronto
            if dado[6].isdigit():
                 ag_naopronto = int(dado[6])
            else:                
                 ag_naopronto = 0
            
            agentes_treinando= int(operadores.count('TREINAMENTO'))  # EM TREINAMENTO
            # tratativa para desconsiderar Sara, Vini, Jessica e operadores em treinamento
            operacoes_ativas = sum(op in op_on for op in ['9902337', '9901194', '9901201'])
            # Calcula as variáveis de acordo com os operadores ativos
            disponiveis = ag_disponiveis - operacoes_ativas - agentes_treinando
            pausa = ag_pausa - operacoes_ativas - agentes_treinando
            
            if check_relatorio < 1:
                # salvar em um arquivo
                data_atual= datetime.now()
                dia_atual= data_atual.day
                mes_atual= data_atual.month
                horario_atual= data_atual.strftime("%H:%M")
                
                
                teste= ['neutro',horario_atual,ag_disponiveis,disponiveis,ag_pronto,ag_pausa,pausa,ag_ocupado,ag_naopronto]
                
                base_tabela= {
                    'hora':[],
                    'agentes': [],
                    'Agentes (Sem BKO, TREINAMENTO)': [],
                    'pronto': [],
                    'pausa': [],
                    'pausa (Sem BKO, TREINAMENTO)': [],
                    'ocupado': [],
                    'Não pronto': [],
                    
                }       
                df = pd.DataFrame(base_tabela)
                # Nome do arquivo Excel e planilha
                arquivo_excel = f'base {dia_atual}-{mes_atual}.xlsx'
                sheet_name = "Sheet1"
                arquivo_existe = bool(os.path.exists(arquivo_excel))
                if not arquivo_existe:
                    df.to_excel(f'base {dia_atual}-{mes_atual}.xlsx', index=False)            
                # Carregar o arquivo Excel
                wb = load_workbook(arquivo_excel)
                ws = wb[sheet_name]              
                # Encontrar a primeira linha vazia na Coluna 1 (Coluna "A")
                primeira_linha_vazia = None
                coluna_verificar = 1  # Número da coluna (1 = Coluna "A")              
                for row_idx in range(1, ws.max_row + 2):  # Inclui uma linha extra no range
                    if ws.cell(row=row_idx, column=coluna_verificar).value is None:  # Verifica se a célula está vazia
                        primeira_linha_vazia = row_idx
                        break   
                if primeira_linha_vazia:
                    for i in range(1,9):
                        ws.cell(row=primeira_linha_vazia, column=i, value=teste[i])
                        wb.save(arquivo_excel)
                print(horario_atual)
                driver.quit()
                time.sleep(530) # 9  minutos  
        check_relatorio= 0 

        if disponiveis != 0:
            conta = float(pausa / disponiveis)
            resultado = int(conta * 100)  # transforma em porcentagem
        else:
            # Trate o caso em que disponiveis é zero
            resultado = 0  # ou outro valor padrão, ou lance uma exceção personalizada
            hora_atual = datetime.now()
            hora_print = hora_atual.strftime("%H:%M:%S")    
        tabela_agentes= driver.find_element(By.XPATH, '//*[@id="dash1_wrapper"]/div[2]/div')
        driver.execute_script("arguments[0].scrollIntoView(true);", tabela_agentes) #desliza a tea até a "tabela agentes"
        driver.save_screenshot('printMonitor.png') # Captura a tela inteira
        driver.quit()
        args.clear() # Limpa a lista de argumentos
        args.append('window-size=1320,1080') # adicione novos argumentos
        # Configurando as opções do WebDriver novamente
        options = Options() 
        for arg in args:
            options.add_argument(arg)
        driver = webdriver.Edge(options=options)
        action = ActionChains(driver)
        
        # teams 
        
        wait = WebDriverWait(driver, 30)
        
        if resultado <= 25:
            loginTeams()
            buscaChat('#css_selector-chat_teams')
            #exibir_data_hora_atual()
            time.sleep(5)
            caixa_de_texto = wait.until(EC.element_to_be_clickable(( By.XPATH, "//div[contains(@id, 'new-message')]")))
            caixa_de_texto.send_keys('Resultado menor que 25%, operação em ordem. (clapfast) {} Agentes logados {} Prontos {} Pausa {} Ocupados {} Não pronto'.format(disponiveis,ag_pronto,pausa, ag_ocupado, ag_naopronto))
            caixa_de_texto.send_keys('\n')
            print(f"{exibir_data_hora_pausa()}, {resultado}%, {disponiveis}, {ag_pronto}, {pausa}, {ag_ocupado}, {ag_naopronto}")  # Exibe a data/hora e o resultado
            time.sleep(5)
            driver.quit()
        elif resultado > 25:
            loginTeams()
            buscaChat('#css_selector-chat_teams2')
            loadingElement(By.XPATH, "//div[contains(@id, 'new-message')]")
            colarImagem()
            time.sleep(1)
            caixa_de_texto = wait.until(EC.element_to_be_clickable(( By.XPATH, "//div[contains(@id, 'new-message')]")))
            caixa_de_texto.send_keys('{}% da operação em pausa, sendo {} agentes logados e {} em pausa, risco de abandono. {} Prontos, {} Ocupados, {} Não pronto.'.format(resultado,disponiveis,pausa,ag_pronto, ag_ocupado, ag_naopronto))
            caixa_de_texto.send_keys(' (screamingfear)')
            caixa_de_texto.send_keys(' ')
            caixa_de_texto.send_keys('\n')
            print(f"{exibir_data_hora_pausa()}, {resultado}%, {disponiveis}, {ag_pronto}, {pausa}, {ag_ocupado}, {ag_naopronto}")  # Exibe a data/hora e o resultado
            time.sleep(5)
            driver.quit()
            #time.sleep(1080)
    else:
        print("Dentro do intervalo de 00:00 a 08:00, script não será executado.")
        time.sleep(1800)